# Задание 3
## Загрузка и предобработка

In [1]:
import osmium as osm
import pandas as pd
import numpy as np
import svgwrite as svg
import random

class Node:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        self.tags = {}
        self.count = 0
        
class Way:
    def __init__(self):
        self.tag = ""
        self.nodes = []    
        
        
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.nodes = {}
        self.ways = []
        self.hospitals = {}
        
        # Рассчитаем информацию для отрисовки
        self.min_lat = 90
        self.max_lat = -90
        self.min_lon = 180
        self.max_lon = 0
        
    
    def node(self, n):
        x = Node(n.location.lat, n.location.lon)
        
        self.min_lat = min(self.min_lat, n.location.lat)
        self.max_lat = max(self.max_lat, n.location.lat)
        self.min_lon = min(self.min_lon, n.location.lon)
        self.max_lon = max(self.max_lon, n.location.lon)            
        
        is_hospital = False
        for tag in n.tags:
            x.tags[tag.k] = tag.v 
            if tag.v == "clinic" or tag.v == "hospital":
                is_hospital = True
        if is_hospital: 
            x.count+=1
            self.hospitals[n.id] = x
            
        self.nodes[n.id] = x
        
    def way(self, w):
        proceed = False
        for tag in w.tags:
            if tag.k == "highway" and (tag.v == "primary" or tag.v == "secondary" or tag.v == "tertiary" or tag.v == "unclassified"):
                proceed  = True
                way_type = tag.v
        if proceed:
            x = Way()
            x.tag = way_type
            for nd in w.nodes:
                x.nodes.append(nd.ref)
                self.nodes[nd.ref].count+=1 
            self.ways.append(x)    

            
def SqrDistance(a, b):
    return (a.lat - b.lat)**2 + (a.lon - b.lon)**2

    
osmhandler = OSMHandler()
osmhandler.apply_file("Data/xapi_meta.osm") 

#Удаляем промежуточные узлы в путях
for w in osmhandler.ways:
    w.shortened_nodes = []
    for i, x in enumerate(w.nodes):
        if osmhandler.nodes[x].count > 1 or i == 0 or i == len(w.nodes) - 1:
            w.shortened_nodes.append(x) 
        else:
            del osmhandler.nodes[x] 
            
#Удаляем не встретившиеся узлы
for k in osmhandler.nodes.keys():
    if osmhandler.nodes[k].count == 0:
        del osmhandler.nodes[k]


#Добавляем пути к госпиталям
hospitals = [x for i, x in enumerate(osmhandler.hospitals.keys()) if i < 10]
min_dist = [100 for i in xrange(0,10)]
closest = ["" for i in xrange(0,10)]

for n in osmhandler.nodes.keys():
    for i,x in enumerate(hospitals):
        if SqrDistance(osmhandler.nodes[n], osmhandler.hospitals[x]) < min_dist[i] and n != x:
            min_dist[i] = SqrDistance(osmhandler.nodes[n], osmhandler.hospitals[x])
            closest[i] = n
            
for i,x in enumerate(hospitals):
    w = Way()
    w.nodes.append(hospitals[i])
    w.nodes.append(closest[i])
    w.shortened_nodes = w.nodes
    osmhandler.ways.append(w)

## А* для нахождения путей

In [2]:
def Manhattan(a, b):
    return abs(a.lat - b.lat) + abs(a.lon - b.lon)

def Chebishev(a, b):
    return max(abs(a.lat - b.lat), abs(a.lon - b.lon)) 

def Distance(a, b, m='euclid'):
    if m == 'manhattan': 
        return Manhattan(a, b)
    elif m == 'chebishev':
        return Chebishev(a, b)
    else:
        return SqrDistance(a, b)**0.5

from heapq import heappush, heappop
        
def Astar(data, s, t, heuristic='chebishev'):    # s,t - node's ids
    open_list = []    #heap
    closed_list = []
    g = 0
    f = 0 + Distance(data.nodes[s], data.nodes[t], heuristic)
    heappush(open_list, (f, g, [s])) #загружаем узлы в кучу в виде кортежа 
    while len(open_list)>0:
        turple = heappop(open_list)
        p = turple[-1] #выгружаем из минимального кортежа путь
        g = turple[1]
        x = p[-1]
        if x in closed_list:
            continue
        if x == t:
            return (f, p)
        closed_list.append(x)
        #
        for successor in adjacency_list[x]:
            ng = g + SqrDistance(data.nodes[successor], data.nodes[x])
            nf = ng + Distance(data.nodes[successor], data.nodes[t], heuristic)**2
            new_path = p[:]
            new_path.append(successor)
            heappush(open_list, (nf, ng, new_path))
    return []  

## Интерфейс для ввода координат

In [3]:
# input interface
print ('Введите значение в пределах: lat ({0}, {1}); lon ({2}, {3})'.format(osmhandler.min_lat, osmhandler.max_lat, osmhandler.min_lon, osmhandler.max_lon))
lat, lon = map(float, raw_input().split())
start = Node(lat, lon)
osmhandler.nodes[0] = start

# Добавляем путь к стартовой позиции
for n in osmhandler.nodes.keys():
    if SqrDistance(osmhandler.nodes[n], start) < min_dist and n != 0:
        min_dist = SqrDistance(osmhandler.nodes[n], start)
        closest = n
w = Way()
w.nodes.append(0)
w.nodes.append(closest)
w.shortened_nodes = w.nodes
osmhandler.ways.append(w)

Введите значение в пределах: lat (56.6991083, 56.9448682); lon (60.2707188, 60.941436)
56.8 60.65


In [4]:
edge_list = []

for w in osmhandler.ways:
    for n in xrange(len(w.shortened_nodes) - 1):
        x = w.shortened_nodes[n]
        y = w.shortened_nodes[n + 1]
        edge_list.append([x,y])
        
adjacency_list = {}
for n in osmhandler.nodes.keys():
    adjacent_nodes = []
    for e in edge_list:
        if e[0] == n:
            adjacent_nodes.append(e[1])
        if e[1] == n:
            adjacent_nodes.append(e[0])
    adjacency_list[n] = adjacent_nodes

## Поиск гамильтонова цикла методом ближайшего соседа

In [5]:
# nearest neighbour
salesman_path = []
salesman_path_length = 0
seen = set()
current = 0;

for k in hospitals:
    hashed_paths = []
    min_length = 1000
    min_ind = -1
    for i,n in enumerate(hospitals):
        if n == current:
            hashed_paths.append((n, [n]))
            continue
        
        t = Astar(osmhandler, current, n)
                
        if t == []:
            hashed_paths.append((n, [n]))
            continue
        #print(i,t[0])
            
        hashed_paths.append((n, t[-1]))
        if min_length > t[0] and n not in seen:
            min_length = t[0]
            min_ind = i
    
    if min_ind>-1:
        salesman_path.append(hashed_paths[min_ind][-1])
        current = hashed_paths[min_ind][0]
        salesman_path_length += min_length
        seen.add(current)
    else:
        break

t = Astar(osmhandler, current, 0)
salesman_path.append(t[-1])
salesman_path_length += t[0]

## Поиск гамильтонова цикла с помощью остовного дерева

In [63]:
def Dijkstra(data, s):
    dist = {s:0}
    prev = {}
    
    seen = set()
    
    for n in data.nodes.keys():
        if n != s:
            dist[n] = 100 #inf
    
    q = [] #heap
    heappush(q, (0, s))
    
    while len(q)>0:
        turple = heappop(q)
        x = turple[1]
        if x in seen:
            continue
        
        seen.add(x)
        for successor in adjacency_list[x]:
            alt = dist[x] + SqrDistance(data.nodes[x], data.nodes[successor])**0.5
            if alt < dist[successor]:
                dist[successor] = alt
                prev[successor] = x
                heappush(q, (alt, successor))
    
    return prev

def get_path(prev, s, t):
    x = t
    path = []
    while x != s:
        path.append(x)
        if x not in prev.keys():
            break
        x = prev[x]
    path.append(s)
    path.reverse()
    return path

In [ ]:
del hospitals[2]
hospitals.insert(0,0)

# Граф маршрутов
paths = [ Dijkstra(osmhandler, s) for s in hospitals ]
grid = []
for i,prev in enumerate(paths):
    row = []
    start = hospitals[i]
    for n in hospitals:
        if n==start:
            length = 1000
            row.append(length)
            continue
        
        x = n
        length = 0
        while x != start:
            if x not in prev.keys():
                length = 1000
                break
            length += SqrDistance(osmhandler.nodes[x], osmhandler.nodes[prev[x]])**0.5
            x = prev[x]
        row.append(length)
    grid.append(row)

In [27]:
# Строим остовное дерево
seen = set()
tree = dict()

seen.add(0)

for k in range(len(grid)-1):
    min_len = 1000
    min_ind = -1
    parent_ind = 0
    for n in seen:
        for i,x in enumerate(grid[n]):
            if x < min_len and i not in seen:
                min_len = x 
                min_ind = i
                parent_ind = n
    if (parent_ind not in tree.keys()):
        tree[parent_ind] = []
    tree[parent_ind].append(min_ind)
    seen.add(min_ind)
  

In [30]:
salesman_nodes = []

def down(n, tree, path):
    path.append(n)
    if n not in tree.keys():
        return
    
    for child in tree[n]:
        down(child, tree, path)
    return

down(0, tree, salesman_nodes)
salesman_nodes.append(0)

In [71]:
salesman_path1 = []
salesman_path1_length = 0
for i in range(len(salesman_nodes)-1):
    salesman_path1.append(get_path(paths[salesman_nodes[i]], hospitals[salesman_nodes[i]], hospitals[salesman_nodes[i+1]]))
    salesman_path1_length += grid[salesman_nodes[i]][salesman_nodes[i+1]]

print ('Длина гамильтонова цикла (ближайший сосед) = {0}'.format(salesman_path_length)) 
print ('Длина гамильтонова цикла (остовное дерево) = {0}'.format(salesman_path1_length))

Длина гамильтонова цикла (ближайший сосед) = 0.2922399
Длина гамильтонова цикла (остовное дерево) = 0.421053984028


## SVG

In [50]:
df = pd.DataFrame(salesman_path)
df.to_csv('salesman_path.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0,878804561,2661430382,3732735377,3732513226,1393925359,220469234,220469233,220469213,521678895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3957589012,521678895,220469213,220469233,220469234,412224917,175266290,319893997,319893904,175266309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1348970925,432577084,432577978,432575467,186368349,186368671,186368666,186368571,175276975,175276967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4052432419,445428494,341979449,1645456330,176668360,2112395507,192192660,409773017,538716090,411232472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3311187481,176134386,176134389,176134392,569925896,569925214,1277005296,409773023,1507014289,409773017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
bot = osmhandler.min_lat
left = osmhandler.min_lon
up = osmhandler.max_lat
right = osmhandler.max_lon

scale = 1000

dwg = svg.Drawing('map_with_salesman_path.svg')
for e in edge_list:
    x1 = (osmhandler.nodes[e[0]].lat - bot)*scale
    y1 = (osmhandler.nodes[e[0]].lon - left)*scale
    x2 = (osmhandler.nodes[e[1]].lat - bot)*scale
    y2 = (osmhandler.nodes[e[1]].lon - left)*scale
    dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(10, 10, 16, '%'), stroke_width=0.5))

# Рисуем пути
for j,path in enumerate(salesman_path): 
    x0 = (osmhandler.nodes[path[0]].lat - bot)*scale
    y0 = (osmhandler.nodes[path[0]].lon - left)*scale
    dwg.add(svg.text.Text(j,x=[x0], y=[y0]))
    for i in xrange(0, len(path) - 1):
        x1 = (osmhandler.nodes[path[i]].lat - bot)*scale
        y1 = (osmhandler.nodes[path[i]].lon - left)*scale
        x2 = (osmhandler.nodes[path[i+1]].lat - bot)*scale
        y2 = (osmhandler.nodes[path[i+1]].lon - left)*scale
        dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(100 - 10*j , 100 + 10*j, 16, '%'), stroke_width=0.5))

dwg.save()